In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("imdb")

In [3]:
dataset.data

{'train': MemoryMappedTable
 text: string
 label: int64
 ----
 text: [["I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered porn

We're gonna try to look at the data treatment

In [4]:
dataset.map(lambda x: x)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
from transformers import (
        AutoTokenizer,
        AutoModelForTokenClassification,
        DataCollatorForTokenClassification,
        Trainer,
        TrainingArguments,
    )

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForTokenClassification.from_pretrained(
        "gpt2", num_labels=2, ignore_mismatched_sizes=True
    )

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

c:\Users\Keytone\miniconda3\envs\slingshot\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Keytone\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly init

In [ ]:
import modal
import wandb
from modal import SharedVolume

In [ ]:
save_weights= modal.Volume.from_name("save_weights")
# Define the Modal App
app = modal.App(name="sentiment-classification")

# Define the image with necessary dependencies
image = (
    modal.Image.debian_slim()
    .pip_install(
        "torch",
        "transformers",
        "pandas",
        "datasets",
        "wandb",
        "scikit-learn"
    )
)


@app.function(
    image=image,
    volumes={"/data": save_weights},
    secrets=[modal.Secret.from_name("wandb-secret")],
    gpu="any",  # Use GPU for training
    timeout=86400,  # 24 hours
)
def train():
    import os
    import torch
    from torch.utils.data import DataLoader
    from transformers import (
        AutoTokenizer,
        AutoModelForTokenClassification,
        DataCollatorForTokenClassification,
        Trainer,
        TrainingArguments,
    )
    from datasets import load_dataset
    import wandb

    # Initialize Weights & Biases
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project="sentiment-classification", name="gpt2-token-classification")

    # Load the IMDB dataset
    dataset = load_dataset("imdb")

    # Map sentiment labels to numerical values
    # This data is already 0 or 1
    # def encode_labels(example):
    #     example["label"] = 1 if example["label"] == "pos" else 0
    #     return example

    # dataset = dataset.map(encode_labels)

    # Initialize the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForTokenClassification.from_pretrained(
        "gpt2", num_labels=2, ignore_mismatched_sizes=True
    )

    # Tokenize the inputs
    def tokenize_function(examples):
        tokens = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=512,
        )
        labels = []
        for _ in range(len(tokens["input_ids"])):
            labels.append([examples["label"]] * 512)
        tokens["labels"] = labels
        return tokens

    tokenized_datasets = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
    )

    # Set format for PyTorch
    tokenized_datasets.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="steps",
        eval_steps=500,
        logging_steps=100,
        save_steps=500,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        report_to="wandb",
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),
        eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(500)),
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()

    # Save the model
    trainer.save_model("/data/sentiment_model")

    # Finish the wandb run
    wandb.finish()


In [ ]:
with app:
    train()
    # Train the model